In [1]:

import os, sys, glob, subprocess, pathlib
import numpy as np
import pandas as pd
import xarray as xr
from mo_evaluation import mo_evaluate
import mo_evaluation_nonstandard
from MOASMO_parameters import get_parameter_from_Namelist_or_lndin

def read_parameter_csv(file_parameter_list):
    df_calibparam = pd.read_csv(file_parameter_list)

    for c in ['Upper', 'Lower', 'Factor', 'Value']:
        if c in df_calibparam.columns:
            if isinstance(df_calibparam.iloc[0][c], str):
                arr = []
                for i in range(len(df_calibparam)):
                    vi = df_calibparam.iloc[i][c]
                    if ',' in vi:
                        arr.append(np.array(vi.split(',')).astype(np.float64))
                    elif '[' in vi:
                        arr.append(np.array(vi.strip('[]').replace('\n', '').split(), dtype=np.float64))
                    else:
                        try:
                            arr.append(np.array([np.float64(vi)]))
                        except:
                            arr.append(np.array([-99999]))
                df_calibparam[c] = arr

    return df_calibparam

def xmlquery_output(pathCTSM, keyword):
    os.chdir(pathCTSM)
    out = subprocess.run(f'./xmlquery {keyword}', shell=True, capture_output=True)
    out = out.stdout.decode().strip().split(' ')[-1]
    return out


def clone_CTSM_model_case(script_clone, source_modelcase, target_modelcase, target_cimeoutput=''):
    if len(target_cimeoutput) > 0:
        settings = f'--cime-output-root {target_cimeoutput}'
    else:
        settings = ''
    _ = subprocess.run(f"{script_clone} --case {target_modelcase} --clone {source_modelcase} {settings} --keepexe", shell=True)
    os.chdir(target_modelcase)
    _ = subprocess.run('./xmlchange BUILD_COMPLETE=TRUE', shell=True)


def update_CTSM_parameter_ParamFile(param_names, param_values, path_CTSM_case, outfile_newparam):
    # param_names and param_values: list
    file_param_base = ''
    # file_user_nl_clm = f'{path_CTSM_clone}/user_nl_clm'
    # with open(file_user_nl_clm) as f:
    #     for line in f:
    #         line = line.strip()
    #         if line.startswith('paramfile'):
    #             file_param_base = line.split('=')[-1].strip().replace('\'', '')
    if len(file_param_base) == 0:
        infile_lndin = f'{path_CTSM_case}/Buildconf/clmconf/lnd_in'
        with open(infile_lndin, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith('paramfile'):
                    file_param_base = line.split('=')[-1].strip().replace('\'', '')

    ds_param = xr.load_dataset(file_param_base)
    for i in range(len(param_names)):
        pn = param_names[i]
        vnew = param_values[i]
        if not pn in ds_param.data_vars:
            print(f'Error!!! Variable {pn} is not find in parameter file {file_param_base}!!!')
            sys.exit()
        else:
            vold = ds_param[pn].values
            print(f'  -- Updating parameter {pn}: old mean value is {np.nanmean(vold)}. New mean value {np.nanmean(vnew)}')
            if vnew.size==1 and ds_param[pn].values.size>1:
                print(f'Warning! New parameter size=1, but raw parameter size={ds_param[pn].values.size}. Force the mean equal to the new parameter value.')
                m = np.nanmean(ds_param[pn].values)
                ds_param[pn].values = ds_param[pn].values - m + np.squeeze(vnew)
            else:
                ds_param[pn].values = np.squeeze(vnew)
    # write to new parameter file
    ds_param.to_netcdf(outfile_newparam, format='NETCDF3_CLASSIC')

def update_CTSM_parameter_NamelistFile(param_names, param_values, file_namelist):
    if len(param_names) > 0:
        with open(file_namelist, 'a') as f:
            print('# Writing new parameter values')
            for i in range(len(param_names)):
                    _ = f.write(f'{param_names[i]} = {np.squeeze(param_values[i])}\n')
                    print(f'  -- Writing parameter {param_names[i]} to namelist file. New value is {param_values[i]}')


def update_CTSM_parameter_SurfdataFile(param_names, param_values, path_CTSM_case, outfile_newsurfdata):

    file_surfdata = ''
    # file_user_nl_clm = f'{path_CTSM_clone}/user_nl_clm'
    # with open(file_user_nl_clm) as f:
    #     for line in f:
    #         line = line.strip()
    #         if line.startswith('fsurdat'):
    #             file_surfdata = line.split('=')[-1].strip().replace('\'', '')
    if len(file_surfdata) == 0:
        infile_lndin = f'{path_CTSM_case}/Buildconf/clmconf/lnd_in'
        with open(infile_lndin, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith('fsurdat'):
                    file_surfdata = line.split('=')[-1].strip().replace('\'', '')

    ds_surf = xr.load_dataset(file_surfdata)
    for i in range(len(param_names)):
        pn = param_names[i]
        vnew = param_values[i]
        if not pn in ds_surf.data_vars:
            print(f'Error!!! Variable {pn} is not find in parameter file {file_surfdata}!!!')
            sys.exit()
        else:
            vold = ds_surf[pn].values
            print(f'  -- Updating surfdata {pn}: old mean value is {np.nanmean(vold)}. New mean value {np.nanmean(vnew)}')
            if vnew.size==1 and ds_surf[pn].values.size>1:
                print(f'Warning! New surfdata size=1, but raw surfdata size={ds_surf[pn].values.size}. Force the mean equal to the new surfdata value.')
                m = np.nanmean(ds_surf[pn].values)
                ds_surf[pn].values[:] = ds_surf[pn].values - m + np.squeeze(vnew)
            else:
                ds_surf[pn].values[:] = np.squeeze(vnew)

    ds_surf.to_netcdf(outfile_newsurfdata, format='NETCDF3_CLASSIC')


def update_ctsm_parameters(path_CTSM_case, file_parameter_set):

    file_user_nl_clm = f'{path_CTSM_case}/user_nl_clm'
    df_calibparam = pd.read_pickle(file_parameter_set)
    #df_calibparam = read_parameter_csv(file_parameter_set)

    dfi = df_calibparam.loc[df_calibparam['Source'] == 'Namelist']
    if len(dfi) > 0:
        param_names = dfi['Parameter'].values
        param_values = dfi['Value'].values
        update_CTSM_parameter_NamelistFile(param_names, param_values, file_user_nl_clm)

    dfi = df_calibparam.loc[df_calibparam['Source'] == 'Param']
    if len(dfi) > 0:
        outfile_newparam = f'{path_CTSM_case}/updated_parameter.nc'
        param_names = dfi['Parameter'].values
        param_values = dfi['Value'].values
        update_CTSM_parameter_ParamFile(param_names, param_values, path_CTSM_case, outfile_newparam)
        # add to the name list file
        with open(file_user_nl_clm, 'a') as f:
            f.write(f"paramfile='{outfile_newparam}'\n")

    dfi = df_calibparam.loc[df_calibparam['Source'] == 'Surfdata']
    if len(dfi) > 0:
        outfile_newsurfdata = f'{path_CTSM_case}/updated_surfdata.nc'
        param_names = dfi['Parameter'].values
        param_values = dfi['Value'].values
        update_CTSM_parameter_SurfdataFile(param_names, param_values, path_CTSM_case, outfile_newsurfdata)
        # add to the name list file
        with open(file_user_nl_clm, 'a') as f:
            f.write(f"fsurdat='{outfile_newsurfdata}'\n")

    print('Successfully update parameters!')

In [3]:
path_CTSM_case='/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Calib_all_HH_MOASMO/level1_100'
file_parameter_set = '/glade/scratch/guoqiang/testparam/paramset_iter0_trial2.pkl'
update_ctsm_parameters(path_CTSM_case, file_parameter_set)

# Writing new parameter values
  -- Writing parameter baseflow_scalar to namelist file. New value is 0.07885625
  -- Writing parameter interception_fraction to namelist file. New value is 0.80625
  -- Writing parameter precip_repartition_nonglc_all_rain_t to namelist file. New value is 1.43
  -- Writing parameter precip_repartition_glc_all_rain_t to namelist file. New value is -0.5700000000000001
  -- Writing parameter precip_repartition_glc_all_snow_t to namelist file. New value is -2.5700000000000003
  -- Writing parameter precip_repartition_nonglc_all_snow_t to namelist file. New value is -0.5700000000000001
  -- Updating parameter lmrse: old mean value is 490.0. New mean value 404.25
  -- Updating parameter fff: old mean value is 0.5. New mean value 4.530475
  -- Updating parameter jmaxb0: old mean value is 0.0311. New mean value 0.0465
  -- Updating parameter sucsat_sf: old mean value is 1.0. New mean value 9.77725
  -- Updating parameter vcmaxse_sf: old mean value is 1.0. New mea

In [12]:
dsp = xr.load_dataset('/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Calib_all_HH_MOASMO/level1_100/updated_parameter.nc')
dsp

<xarray.Dataset>
Dimensions:                           (pft: 79, ndecomp_pools_max: 8,
                                       segment: 4, variants: 2)
Coordinates:
    pftname                           (pft) |S40 b'not_vegetated             ...
  * segment                           (segment) |S40 b'sunlit                ...
    variantnames                      (variants) object b'water              ...
Dimensions without coordinates: pft, ndecomp_pools_max, variants
Data variables: (12/402)
    C2_liq_Brun89                     float64 4.22e-13
    FUN_fracfixers                    (pft) float64 0.0 0.3195 ... 1.278 1.278
    a_coef                            float64 0.13
    a_exp                             float64 0.45
    a_fix                             (pft) float64 -3.62 -3.62 ... -3.62 -3.62
    accum_factor                      float64 0.1
    ...                                ...
    zsno                              float64 0.002985
    ztopmx                            (pft) float64 nan nan nan ... 2.5 1.0 1.0
    mimics_fchem_k                    (segment) float64 0.3 -3.0 3.0 0.0
    mimics_fchem_r                    (segment) float64 0.1 -3.0 3.0 0.0
    mimics_initial_Cstocks            (ndecomp_pools_max) float64 1.0 ... 1.0
    mimics_initial_Cstocks_depth      float64 1.5
Attributes:
    history:                    Fri Nov 12 12:18:20 2021: ncks -A -v mimics_f...
    Conventions:                CF-1.0
    title:                      Vegetation (Plant Function Type or PFT) const...
    NCO:                        netCDF Operators version 4.9.5 (Homepage = ht...
    nco_openmp_thread_number:   1
    history_of_appended_files:  Fri Nov 12 12:18:20 2021: Appended file ctsm5...

In [10]:
df=pd.read_pickle('/glade/scratch/guoqiang/testparam/paramset_iter0_trial2.pkl')
df.loc[8]['Value'].mean()

5.035636750000002e-09

In [8]:
dsp.kmax.values

array([[0.00000000e+00, 4.21460019e-09, 5.87828691e-09, 4.70891858e-09,
        8.72645548e-09, 6.84663820e-09, 1.05956176e-08, 6.52236771e-09,
        7.18703035e-09, 3.02177403e-09, 6.62226100e-10, 1.08327218e-09,
        6.35249737e-10, 5.26825023e-10, 3.40421905e-10, 5.40912703e-09,
        5.26121416e-09, 5.26121416e-09, 5.26121416e-09, 5.26121416e-09,
        5.26121416e-09, 5.26121416e-09, 5.26121416e-09, 5.26121416e-09,
        5.26121416e-09, 5.26121416e-09, 5.26121416e-09, 5.26121416e-09,
        5.26121416e-09, 5.26121416e-09, 5.26121416e-09, 5.26121416e-09,
        5.26121416e-09, 5.26121416e-09, 5.26121416e-09, 5.26121416e-09,
        5.26121416e-09, 5.26121416e-09, 5.26121416e-09, 5.26121416e-09,
        5.26121416e-09, 5.26121416e-09, 5.26121416e-09, 5.26121416e-09,
        5.26121416e-09, 5.26121416e-09, 5.26121416e-09, 5.26121416e-09,
        5.26121416e-09, 5.26121416e-09, 5.26121416e-09, 5.26121416e-09,
        5.26121416e-09, 5.26121416e-09, 5.26121416e-09, 5.261214